In [ ]:
import pandas as pd
import mlflow
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import logging

In [26]:
# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class SentimentValidator:
    def __init__(self):
        self.vectorizer = None
        self.label_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
        self.inverse_mapping = {v: k for k, v in self.label_mapping.items()}

    def load_data(self, parquet_path):
        """Carrega e transforma os dados de validação"""
        try:
            df = pd.read_parquet(parquet_path)
            
            # Verificação de colunas essenciais
            if not all(col in df.columns for col in ['comment_cleaned', 'sentiment']):
                raise ValueError("Colunas 'comment_cleaned' ou 'sentiment' não encontradas")
            
            # Filtragem e limpeza
            df = df[df['sentiment'].isin(self.label_mapping.keys())]
            df = df.dropna(subset=['comment_cleaned', 'sentiment'])
            
            if len(df) == 0:
                raise ValueError("Nenhum dado válido após filtragem")
                
            return df['comment_cleaned'].values, df['sentiment'].values
        
        except Exception as e:
            logger.error(f"Erro ao carregar dados: {str(e)}")
            raise

    def load_model_and_components(self, model_name):
        """Carrega o modelo e extrai o vetorizador corretamente"""
        try:
            model_uri = f"models:/sentiment_{model_name}/latest"
            
            sklearn_model = mlflow.sklearn.load_model(model_uri)
            return sklearn_model
        
        except Exception as e:
            logger.error(f"Erro ao carregar modelo {model_name}: {str(e)}")
            raise

    def transform_data(self, X):
        """Transforma os dados conforme o pipeline de treinamento"""
        if self.vectorizer is None:
            logger.warning("Vetorizador não encontrado, criando novo como fallback")
            self.vectorizer = TfidfVectorizer(max_features=5000)
            
            # Apenas fit se for um novo vetorizador (evitar data leakage)
            self.vectorizer.fit(X)
        
        return self.vectorizer.transform(X)

    def validate(self, model_name="randomforest", parquet_path="../data/dataset_valid_with_sentiment.parquet"):
        """Executa a validação completa"""
        try:
            with mlflow.start_run(run_name=f"Validation_{model_name}"):
                # 1. Carregar dados
                X_val, y_val_true = self.load_data(parquet_path)
                logger.info(f"Dados carregados: {len(X_val)} amostras")
                
                # 2. Carregar modelo e componentes
                model = self.load_model_and_components(model_name)
                
                # 3. Fazer previsões diretamente (o modelo já inclui o pipeline completo)
                y_val_pred = model.predict(X_val)
                
                # 4. Converter labels numéricos para texto se necessário
                if all(isinstance(x, (int, float, np.integer)) for x in y_val_pred):
                    y_val_pred = [self.inverse_mapping.get(int(x), 'Neutral') for x in y_val_pred]
                
                # 5. Calcular métricas
                self._log_metrics(y_val_true, y_val_pred, model_name)
                
                return True
                
        except Exception as e:
            logger.error(f"Falha na validação: {str(e)}")
            return False

    def _log_metrics(self, y_true, y_pred, model_name):
        """Calcula e registra métricas no MLflow"""
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='weighted')
        report = classification_report(y_true, y_pred, output_dict=True)
        
        # Log básico
        mlflow.log_metrics({
            "val_accuracy": accuracy,
            "val_f1_weighted": f1
        })
        
        # Log por classe
        for cls in ['Negative', 'Neutral', 'Positive']:
            if cls in report:
                mlflow.log_metrics({
                    f"val_precision_{cls.lower()}": report[cls]['precision'],
                    f"val_recall_{cls.lower()}": report[cls]['recall'],
                    f"val_f1_{cls.lower()}": report[cls]['f1-score'],
                    f"val_support_{cls.lower()}": report[cls]['support']
                })
        
        # Matriz de confusão
        self._plot_confusion_matrix(y_true, y_pred, model_name)
        
        logger.info(f"\nModelo: {model_name}")
        logger.info(f"Acurácia: {accuracy:.4f}")
        logger.info(f"F1-Score: {f1:.4f}")
        logger.info("\nRelatório de Classificação:")
        logger.info(classification_report(y_true, y_pred))

    def _plot_confusion_matrix(self, y_true, y_pred, model_name):
        """Gera e salva a matriz de confusão"""
        cm = confusion_matrix(y_true, y_pred, labels=['Negative', 'Neutral', 'Positive'])
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Negative', 'Neutral', 'Positive'],
                    yticklabels=['Negative', 'Neutral', 'Positive'])
        plt.title(f'Matriz de Confusão - {model_name}')
        plt.ylabel('Verdadeiro')
        plt.xlabel('Previsto')
        
        cm_path = f"confusion_matrix_{model_name}.png"
        plt.savefig(cm_path)
        mlflow.log_artifact(cm_path)
        plt.close()

In [27]:
# Configuração
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment("Restaurant_Sentiment_Validation")

validator = SentimentValidator()

# Lista de modelos para validar
models_to_validate = ['randomforest', 'logisticregression']

for model_name in models_to_validate:
    logger.info(f"\nIniciando validação para {model_name}...")
    success = validator.validate(model_name)
    
    if success:
        logger.info(f"Validação de {model_name} concluída com sucesso!")
    else:
        logger.info(f"Validação de {model_name} falhou.")

INFO:__main__:
Iniciando validação para randomforest...


INFO:__main__:Dados carregados: 199 amostras


INFO:__main__:
Modelo: randomforest
INFO:__main__:Acurácia: 0.6985
INFO:__main__:F1-Score: 0.6437
INFO:__main__:
Relatório de Classificação:
INFO:__main__:              precision    recall  f1-score   support

    Negative       0.70      0.42      0.53        50
     Neutral       0.33      0.04      0.07        25
    Positive       0.70      0.94      0.81       124

    accuracy                           0.70       199
   macro avg       0.58      0.47      0.47       199
weighted avg       0.66      0.70      0.64       199

INFO:__main__:Validação de randomforest concluída com sucesso!
INFO:__main__:
Iniciando validação para logisticregression...
INFO:__main__:Dados carregados: 199 amostras


🏃 View run Validation_randomforest at: http://127.0.0.1:5000/#/experiments/768184713491958936/runs/ed4f45879c4548068e78f3a4b1fbe18a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/768184713491958936


INFO:__main__:
Modelo: logisticregression
INFO:__main__:Acurácia: 0.6884
INFO:__main__:F1-Score: 0.6068
INFO:__main__:
Relatório de Classificação:
INFO:__main__:              precision    recall  f1-score   support

    Negative       0.92      0.24      0.38        50
     Neutral       0.50      0.04      0.07        25
    Positive       0.67      1.00      0.81       124

    accuracy                           0.69       199
   macro avg       0.70      0.43      0.42       199
weighted avg       0.71      0.69      0.61       199

INFO:__main__:Validação de logisticregression concluída com sucesso!


🏃 View run Validation_logisticregression at: http://127.0.0.1:5000/#/experiments/768184713491958936/runs/755fe2a5a3174e619d022fa7a5831967
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/768184713491958936
